In [1]:
import os
os.environ['PYTHONHASHSEED']=str(0)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import random
random.seed(0)
import numpy as np
np.random.seed(0)
import tensorflow as tf
tf.compat.v1.set_random_seed(0)
import matplotlib.pyplot as plt
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
#tf.compat.v1.keras.backend.set_session(sess)
import pickle
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.utils import to_categorical
import keras.initializers
import csv
MNIST_image_path = 'MNIST_data'


Using TensorFlow backend.


In [2]:
def load_MNIST_data(MNIST_image_path, filename):
    
    features = []
    labels = []
    
    with open(MNIST_image_path + '/' + filename + '.csv') as file:
        file_csv = csv.reader(file)
        
        next(file_csv, None)
        for row in file_csv:
        
            labels.append(int(row[0]))
            features.append(row[1:785])
            
    features = np.array(features)
    labels = np.array(labels)
    features = features.astype(np.float)
    print('Data size is {}'.format(features.shape))
    features = features.reshape(len(features), 28, 28)
    
    return features, labels

def load_MNIST_test_data(MNIST_image_path, filename):
    
    features = []
    labels = []
    
    with open(MNIST_image_path + '/' + filename + '.csv') as file:
        file_csv = csv.reader(file)
        
        next(file_csv, None)
        for row in file_csv:
        
            labels.append(int(row[0]))
            features.append(row[1:785])
            
    features = np.array(features)
    labels = np.array(labels)
    features = features.astype(np.float)
    print('Data size is {}'.format(features.shape))
    features = features.reshape(len(features), 28, 28)
    
    return features, labels
    

In [3]:
def create_model(learning_rate):
    # dropout in each layer
    # learning rate
    
    #dropout = trial.suggest_uniform("dropout", 0.1, 0.5)
    #dropout = 0.2
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64,(3, 3)))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    
    #model.add(Dropout(rate=dropout))
    model.add(Dense(512))
    model.add(Activation('relu'))

    #model.add(Dropout(rate=dropout))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    lr = learning_rate
    
    opt = SGD(lr=lr)

    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

In [4]:
training_images, training_labels = load_MNIST_data(MNIST_image_path, 'train')
training_images = training_images.reshape(training_images.shape[0], 28, 28, 1)
training_images = training_images/255.0
    
test_images, test_labels = load_MNIST_test_data(MNIST_image_path, 'test')
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)
test_images = test_images/255.0

def update(learning_rate, epsilon):
    
    print('Not using data augmentation.')
    
    model1 = create_model(learning_rate)
    model1.load_weights('base_model_optuna.h5')
    
    # Fit the model on the training data.
    
    history1 = model1.fit(training_images, training_labels,
              epochs=6, callbacks=None,
              validation_split = 0.1, verbose = 2, shuffle = False)
    

    # Evaluate the model accuracy on the test set.
    score1 = model1.evaluate(test_images, test_labels, verbose=0)
    
    tf.keras.backend.clear_session()
    
    # Generate our trial model.
    model2 = create_model(learning_rate + epsilon)
    model2.load_weights('base_model_optuna.h5')
    
    # Fit the model on the training data.
    
    history2 = model2.fit(training_images, training_labels, 
              epochs=6, callbacks=None,
              validation_split = 0.1, verbose = 2, shuffle = False)
    

    # Evaluate the model accuracy on the test set.
    score2 = model2.evaluate(test_images, test_labels, verbose=0)
    
    tf.keras.backend.clear_session()
    
    model3 = create_model(learning_rate - epsilon)
    model3.load_weights('base_model_optuna.h5')
    
    # Fit the model on the training data.

    history3 = model3.fit(training_images, training_labels, 
              epochs=6, callbacks=None,
              validation_split = 0.1, verbose = 2, shuffle = False)
    
    # Evaluate the model accuracy on the test set.
    score3 = model3.evaluate(test_images, test_labels, verbose=0)
    
    tf.keras.backend.clear_session()
    
    del model1
    del model2
    del model3
    
    print ('Test loss: {}, Test accuracy: {}'.format(score1[0], score1[1]*100.0))
    print ('Test loss: {}, Test accuracy: {}'.format(score2[0], score2[1]*100.0))
    print ('Test loss: {}, Test accuracy: {}'.format(score3[0], score3[1]*100.0))
    

    first_der = (score2[1] - score3[1])/(2*epsilon)
    second_der = (score2[1] + score3[1] - (2*score1[1]))/(epsilon*epsilon)
    learning_rate = learning_rate + (first_der/second_der)
    epsilon = 0.001*learning_rate
    
    return learning_rate, score1[1], score2[1], score3[1], epsilon



Data size is (60000, 784)
Data size is (10000, 784)


In [5]:
global learning_rate
learning_rate = 0.09554754641960188
global epsilon
epsilon = 9.554754641960188e-05
fx = []
fxminh = []
fxplush = []
result = []
acc = []


for i in range(10):
    lr, x, y, z, ac = update(learning_rate,epsilon)
    learning_rate = lr
    print('Updated learning rate is {}',format(learning_rate))
    fx.append(x)
    fxminh.append(y)
    fxplush.append(z)
    result.append(lr)
    acc.append(ac)
    print('Next step calculating................')
    

Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
activation_3 (Activation)

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 32s - loss: 0.1656 - accuracy: 0.9490 - val_loss: 0.0620 - val_accuracy: 0.9808
Epoch 2/6
 - 32s - loss: 0.0421 - accuracy: 0.9871 - val_loss: 0.0404 - val_accuracy: 0.9888
Epoch 3/6
 - 32s - loss: 0.0261 - accuracy: 0.9921 - val_loss: 0.0413 - val_accuracy: 0.9890
Epoch 4/6
 - 32s - loss: 0.0183 - accuracy: 0.9941 - val_loss: 0.0390 - val_accuracy: 0.9900
Epoch 5/6
 - 32s - loss: 0.0134 - accuracy: 0.9958 - val_loss: 0.0418 - val_accuracy: 0.9880
Epoch 6/6
 - 32s - loss: 0.0108 - accuracy: 0.9965 - val_loss: 0.0476 - val_accuracy: 0.9882
Test loss: 0.04630223853733351, Test accuracy: 98.73999953269958
Test loss: 0.040858919674631164, Test accuracy: 98.91999959945679
Test loss: 0.05012474996557075, Test accuracy: 98.72000217437744
Updated learning rate is {} 0.09560726185643252
Next step calculating................
Not using data augmentation.
Model: "sequential_1"
____________________________________________________________

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 33s - loss: 0.1620 - accuracy: 0.9499 - val_loss: 0.0600 - val_accuracy: 0.9822
Epoch 2/6
 - 34s - loss: 0.0428 - accuracy: 0.9873 - val_loss: 0.0393 - val_accuracy: 0.9890
Epoch 3/6
 - 34s - loss: 0.0262 - accuracy: 0.9921 - val_loss: 0.0350 - val_accuracy: 0.9898
Epoch 4/6
 - 33s - loss: 0.0180 - accuracy: 0.9943 - val_loss: 0.0352 - val_accuracy: 0.9913
Epoch 5/6
 - 33s - loss: 0.0151 - accuracy: 0.9949 - val_loss: 0.0325 - val_accuracy: 0.9922
Epoch 6/6
 - 34s - loss: 0.0104 - accuracy: 0.9966 - val_loss: 0.0364 - val_accuracy: 0.9930
Test loss: 0.036268734660002716, Test accuracy: 99.04999732971191
Test loss: 0.038154960406963576, Test accuracy: 99.00000095367432
Test loss: 0.03434146589120446, Test accuracy: 99.08000230789185
Updated learning rate is {} 0.09579844241219339
Next step calculating................
Not using data augmentation.
Model: "sequential_1"
___________________________________________________________

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 32s - loss: 0.1641 - accuracy: 0.9492 - val_loss: 0.0528 - val_accuracy: 0.9843
Epoch 2/6
 - 32s - loss: 0.0428 - accuracy: 0.9870 - val_loss: 0.0400 - val_accuracy: 0.9880
Epoch 3/6
 - 31s - loss: 0.0263 - accuracy: 0.9921 - val_loss: 0.0412 - val_accuracy: 0.9878
Epoch 4/6
 - 31s - loss: 0.0182 - accuracy: 0.9943 - val_loss: 0.0352 - val_accuracy: 0.9908
Epoch 5/6
 - 31s - loss: 0.0141 - accuracy: 0.9956 - val_loss: 0.0387 - val_accuracy: 0.9898
Epoch 6/6
 - 32s - loss: 0.0108 - accuracy: 0.9965 - val_loss: 0.0380 - val_accuracy: 0.9918
Test loss: 0.041823688757342624, Test accuracy: 98.89000058174133
Test loss: 0.036355287044364686, Test accuracy: 98.94000291824341
Test loss: 0.03838121354798164, Test accuracy: 98.94999861717224
Updated learning rate is {} 0.0957941012245655
Next step calculating................
Not using data augmentation.
Model: "sequential_1"
____________________________________________________________

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 33s - loss: 0.1624 - accuracy: 0.9500 - val_loss: 0.0572 - val_accuracy: 0.9828
Epoch 2/6
 - 33s - loss: 0.0426 - accuracy: 0.9866 - val_loss: 0.0399 - val_accuracy: 0.9892
Epoch 3/6
 - 32s - loss: 0.0272 - accuracy: 0.9919 - val_loss: 0.0349 - val_accuracy: 0.9902
Epoch 4/6
 - 32s - loss: 0.0181 - accuracy: 0.9945 - val_loss: 0.0359 - val_accuracy: 0.9893
Epoch 5/6
 - 33s - loss: 0.0149 - accuracy: 0.9949 - val_loss: 0.0464 - val_accuracy: 0.9883
Epoch 6/6
 - 33s - loss: 0.0112 - accuracy: 0.9963 - val_loss: 0.0361 - val_accuracy: 0.9908
Test loss: 0.0440730098653439, Test accuracy: 98.86999726295471
Test loss: 0.033272118152700356, Test accuracy: 99.11999702453613
Test loss: 0.04006710385514532, Test accuracy: 98.90000224113464
Updated learning rate is {} 0.09583163623521752
Next step calculating................
Not using data augmentation.
Model: "sequential_1"
_____________________________________________________________

...

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 36s - loss: 0.1705 - accuracy: 0.9483 - val_loss: 0.0535 - val_accuracy: 0.9852
Epoch 2/6
 - 35s - loss: 0.0424 - accuracy: 0.9874 - val_loss: 0.0416 - val_accuracy: 0.9880
Epoch 3/6
 - 35s - loss: 0.0257 - accuracy: 0.9924 - val_loss: 0.0404 - val_accuracy: 0.9892
Epoch 4/6
 - 35s - loss: 0.0184 - accuracy: 0.9942 - val_loss: 0.0387 - val_accuracy: 0.9900
Epoch 5/6
 - 36s - loss: 0.0137 - accuracy: 0.9957 - val_loss: 0.0383 - val_accuracy: 0.9922
Epoch 6/6
 - 35s - loss: 0.0103 - accuracy: 0.9967 - val_loss: 0.0361 - val_accuracy: 0.9915
Test loss: 0.036763226169922655, Test accuracy: 99.04999732971191
Test loss: 0.04003099655740706, Test accuracy: 98.86000156402588
Test loss: 0.040528983614335774, Test accuracy: 98.90999794006348
Updated learning rate is {} 0.09622105733427849
Next step calculating................
Not using data augmentation.
Model: "sequential_1"
___________________________________________________________

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 33s - loss: 0.1583 - accuracy: 0.9509 - val_loss: 0.0620 - val_accuracy: 0.9827
Epoch 2/6
 - 33s - loss: 0.0423 - accuracy: 0.9873 - val_loss: 0.0384 - val_accuracy: 0.9898
Epoch 3/6
 - 33s - loss: 0.0262 - accuracy: 0.9919 - val_loss: 0.0391 - val_accuracy: 0.9892
Epoch 4/6
 - 33s - loss: 0.0175 - accuracy: 0.9945 - val_loss: 0.0341 - val_accuracy: 0.9908
Epoch 5/6
 - 33s - loss: 0.0133 - accuracy: 0.9959 - val_loss: 0.0461 - val_accuracy: 0.9892
Epoch 6/6
 - 32s - loss: 0.0112 - accuracy: 0.9964 - val_loss: 0.0359 - val_accuracy: 0.9920
Test loss: 0.04077626323496009, Test accuracy: 98.9799976348877
Test loss: 0.03623641024353224, Test accuracy: 99.01999831199646
Test loss: 0.033072300725049374, Test accuracy: 99.04999732971191
Updated learning rate is {} 0.09620802859406907
Next step calculating................
Not using data augmentation.
Model: "sequential_1"
_____________________________________________________________

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 31s - loss: 0.1631 - accuracy: 0.9492 - val_loss: 0.0588 - val_accuracy: 0.9833
Epoch 2/6
 - 32s - loss: 0.0420 - accuracy: 0.9874 - val_loss: 0.0394 - val_accuracy: 0.9897
Epoch 3/6
 - 31s - loss: 0.0263 - accuracy: 0.9922 - val_loss: 0.0409 - val_accuracy: 0.9892
Epoch 4/6
 - 32s - loss: 0.0188 - accuracy: 0.9941 - val_loss: 0.0330 - val_accuracy: 0.9908
Epoch 5/6
 - 31s - loss: 0.0145 - accuracy: 0.9954 - val_loss: 0.0364 - val_accuracy: 0.9910
Epoch 6/6
 - 31s - loss: 0.0102 - accuracy: 0.9968 - val_loss: 0.0506 - val_accuracy: 0.9882
Test loss: 0.04940942265447468, Test accuracy: 98.65000247955322
Test loss: 0.036953214032203856, Test accuracy: 98.989999294281
Test loss: 0.04323951276144871, Test accuracy: 98.9300012588501
Updated learning rate is {} 0.09621265174391938
Next step calculating................
Not using data augmentation.
Model: "sequential_1"
_______________________________________________________________

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 29s - loss: 0.1625 - accuracy: 0.9504 - val_loss: 0.0562 - val_accuracy: 0.9835
Epoch 2/6
 - 27s - loss: 0.0424 - accuracy: 0.9871 - val_loss: 0.0399 - val_accuracy: 0.9893
Epoch 3/6
 - 28s - loss: 0.0260 - accuracy: 0.9921 - val_loss: 0.0366 - val_accuracy: 0.9900
Epoch 4/6
 - 28s - loss: 0.0183 - accuracy: 0.9946 - val_loss: 0.0407 - val_accuracy: 0.9893
Epoch 5/6
 - 29s - loss: 0.0142 - accuracy: 0.9954 - val_loss: 0.0407 - val_accuracy: 0.9905
Epoch 6/6
 - 27s - loss: 0.0107 - accuracy: 0.9964 - val_loss: 0.0425 - val_accuracy: 0.9898
Test loss: 0.03481864907849458, Test accuracy: 99.09999966621399
Test loss: 0.033243644426437194, Test accuracy: 99.09999966621399
Test loss: 0.04270228496606196, Test accuracy: 98.8099992275238
Updated learning rate is {} 0.09616487797070958
Next step calculating................
Not using data augmentation.
Model: "sequential_1"
_____________________________________________________________

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
 - 31s - loss: 0.1629 - accuracy: 0.9503 - val_loss: 0.0601 - val_accuracy: 0.9825
Epoch 2/6
 - 18s - loss: 0.0429 - accuracy: 0.9872 - val_loss: 0.0418 - val_accuracy: 0.9890
Epoch 3/6
 - 30s - loss: 0.0264 - accuracy: 0.9919 - val_loss: 0.0378 - val_accuracy: 0.9890
Epoch 4/6
 - 31s - loss: 0.0181 - accuracy: 0.9944 - val_loss: 0.0390 - val_accuracy: 0.9893
Epoch 5/6
 - 31s - loss: 0.0142 - accuracy: 0.9951 - val_loss: 0.0324 - val_accuracy: 0.9918
Epoch 6/6
 - 31s - loss: 0.0116 - accuracy: 0.9964 - val_loss: 0.0344 - val_accuracy: 0.9927
Test loss: 0.03829944274430759, Test accuracy: 98.71000051498413
Test loss: 0.04300601608338611, Test accuracy: 98.7999975681305
Test loss: 0.032112942201695, Test accuracy: 99.15000200271606
Updated learning rate is {} 0.09613332876596534
Next step calculating................


In [6]:
fx = np.array(fx)
fxminh = np.array(fxminh)
fxplush = np.array(fxplush)
result = np.array(result)
acc = np.array(acc)
for i in range(len(fx)):
    print(fx[i])
for i in range(len(fxminh)):
    print(fxminh[i])
for i in range(len(fxplush)):
    print(fxplush[i])
for i in range(len(result)):
    print(result[i])
for i in range(len(acc)):
    print(acc[i])


0.9873999953269958
0.9904999732971191
0.9889000058174133
0.9886999726295471
0.989300012588501
0.9904999732971191
0.989799976348877
0.9865000247955322
0.9909999966621399
0.9871000051498413
0.9891999959945679
0.9900000095367432
0.9894000291824341
0.9911999702453613
0.991100013256073
0.9886000156402588
0.9901999831199646
0.9898999929428101
0.9909999966621399
0.9879999756813049
0.9872000217437744
0.9908000230789185
0.9894999861717224
0.9890000224113464
0.9879000186920166
0.9890999794006348
0.9904999732971191
0.989300012588501
0.988099992275238
0.9915000200271606
0.09560726185643252
0.09579844241219339
0.0957941012245655
0.09583163623521752
0.09621381930216948
0.09622105733427849
0.09620802859406907
0.09621265174391938
0.09616487797070958
0.09613332876596534
9.560726185643253e-05
9.57984424121934e-05
9.57941012245655e-05
9.583163623521753e-05
9.621381930216948e-05
9.622105733427849e-05
9.620802859406907e-05
9.621265174391938e-05
9.616487797070959e-05
9.613332876596533e-05
